<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the datasets
train_df = pd.read_csv('/content/preprocessed_train.csv')
val_df = pd.read_csv('/content/prerocessed_validation (1).csv')

# Extract the `phrases` column
train_sentences = train_df['phrases'].tolist()
val_sentences = val_df['phrases'].tolist()

In [5]:
from transformers import DistilBertTokenizer
import random

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

def mask_text(sentences, tokenizer, mask_probability=0.15):
    inputs, labels = [], []
    for sentence in sentences:
        # Tokenize the sentence
        tokenized = tokenizer(sentence, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

        # Create labels and randomly mask tokens
        input_ids = tokenized["input_ids"][0]
        label_ids = input_ids.clone()

        for i in range(len(input_ids)):
            if random.random() < mask_probability and input_ids[i] != tokenizer.pad_token_id:
                input_ids[i] = tokenizer.mask_token_id  # Replace token with [MASK]
            else:
                label_ids[i] = -100  # Ignore token for loss calculation

        inputs.append(input_ids)
        labels.append(label_ids)

    return {"input_ids": inputs, "labels": labels}

# Prepare datasets
train_data = mask_text(train_sentences, tokenizer)
val_data = mask_text(val_sentences, tokenizer)

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

class HinglishDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx],
            "labels": self.labels[idx],
        }

# Create datasets
train_dataset = HinglishDataset(train_data["input_ids"], train_data["labels"])
val_dataset = HinglishDataset(val_data["input_ids"], val_data["labels"])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [7]:
from transformers import DistilBertForMaskedLM, AdamW
from transformers import get_scheduler
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Load model
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-multilingual-cased')

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # Assume 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=

In [ ]:
# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Epoch 0:  93%|█████████▎| 10974/11783 [1:30:17<06:38,  2.03it/s, loss=1.14]

In [ ]:
model.save_pretrained('./hinglish_next_word_model')
tokenizer.save_pretrained('./hinglish_next_word_model')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
fill_mask = pipeline('fill-mask', model='./hinglish_next_word_model', tokenizer='./hinglish_next_word_model')

# Test with an example
text = "Mujhe lagta hai ki [MASK]"
predictions = fill_mask(text)

for pred in predictions:
    print(f"Suggested word: {pred['token_str']} with score: {pred['score']}")

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
fill_mask = pipeline('fill-mask', model='./hinglish_next_word_model', tokenizer='./hinglish_next_word_model')

def predict_next_word():
    print("Hinglish Next-Word Prediction")
    print("Type a word or sentence to predict the next word.")
    print("Type 'exit' to quit.")

    while True:
        user_input = input("Enter a word or sentence: ").strip()
        if user_input.lower() == 'exit':
            print("Exiting. Goodbye!")
            break

        # Add the [MASK] token at the end of the input
        input_with_mask = f"{user_input} [MASK]"

        # Get predictions from the model
        try:
            predictions = fill_mask(input_with_mask)
            print("\nPredicted next words:")
            for i, pred in enumerate(predictions):
                print(f"{i+1}. {pred['token_str']} (score: {pred['score']:.4f})")
        except Exception as e:
            print("Error during prediction:", e)
        print()

# Run the prediction function
predict_next_word()